# 智能RSI参数

根据个股一年来最低的RSI来判断当前股价是否处于最低位置

In [20]:
from alpha.notebook import *
import traceback

from alpha.core.rsi_stats import RsiStats, rsiday, rsi30
await init_notebook(use_omicron=True)

In [ ]:
# rsi = RsiStats(FrameType.MIN30)
# await rsi.calc("2021-09-30 15:00")

In [ ]:
# rsiday = RsiStats(FrameType.DAY)
# await rsiday.calc("2021-09-30")

In [21]:
async def predict_xshg():
    xshg = "000001.XSHG"
    end = arrow.now()
    start = tf.shift(tf.floor(end, FrameType.MIN30), -39, FrameType.MIN30)
    
    bars = await Security(xshg).load_bars(start, end, FrameType.MIN30)
    rsi_now = relative_strength_index(bars["close"], 6)[-1]
    rsi_max = rsi30.get_rsi(xshg, 0.9)
    rsi_min = rsi30.get_rsi(xshg, 0.1)
    
    if rsi_max - rsi_now < 5:
        print("上证rsi距高位不足5个点")
        
    if rsi_now - rsi_min < 5:
        print("上证rsi距低位不足5个点")

In [22]:
await predict_xshg()

In [25]:
# 格林美
print(f'{rsiday.get_proba("002340.XSHE", 26.59):.2%}')

8.70%


In [24]:
# 格林美
rsi30.get_proba("002340.XSHE", 24.25)

0.09584584584584585